In [1]:
from wordcount_utils import removePunctuation
from time import process_time
import pandas as pd
import numpy as np
from scipy.stats import mode
from sklearn.datasets import make_blobs
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import make_scorer, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score, train_test_split, LeaveOneOut, GridSearchCV, StratifiedKFold, cross_validate
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
import pyspark
from pyspark.ml.feature import FeatureHasher
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, trim, col, lower
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [2]:
#Specify the path of the data file
datafile = '/dse230final/allpep.csv'
# Initialize Spark
spark= pyspark.sql.SparkSession.builder.master('local[*]').appName("DSE Final Project").getOrCreate()

In [3]:
rawdf=spark.read.option("header",True).csv(datafile)
rawdf=rawdf.drop('_c0')
w = Window().orderBy(f.lit('A'))
newdf=rawdf.withColumn("Disease_ID", f.row_number().over(w))
print(newdf.show(3))
print('The dataset contains ' + str(newdf.count()) + ' '

112361
+---------+-----------------+------------+--------+--------------------+------------------+----------------+----------+-----+---------+--------------------+--------+----------+
|Cell Type|View Related Data|Epitope Type|Position|    Peptide Sequence|           Disease|Epitope Sequence|peptide_id|   ID|MHC Types|        Epitope Name|Antibody|Disease_ID|
+---------+-----------------+------------+--------+--------------------+------------------+----------------+----------+-----+---------+--------------------+--------+----------+
|    Bcell|              793|      Simple|       5|QVPLVQQQQFPGQQQPF...|allergy by trigger|      VQQQQFPGQQ|      8501|21969|     null|alpha/beta gliadi...|     IgE|         1|
|    Bcell|              793|      Simple|      23|VPVPQLQPQNPSQQQPQ...|allergy by trigger|      VQQQQFPGQQ|      8506|22035|     null|alpha/beta gliadi...|     IgE|         2|
|    Bcell|              793|      Simple|      23|VPVPQPQPQNPSQPQPQ...|allergy by trigger|      VQQQQFPGQQ|

In [11]:
labelpepdf=newdf.withColumn("Disease_ID_Main", f.when(newdf.Disease.startswith('celiac disease'),
            0).when(newdf.Disease.startswith('allergy'),1).when(newdf.Disease.startswith('healthy'),
            2).when(newdf.Disease.startswith('diabetes'),3).otherwise(4))